In [1]:
import numpy as np
import pandas as pd
import warnings

In [2]:
path = 'D:\\資料處理\\SBAdata\\SBAnational.csv'
df = pd.read_csv(path)
df_copy = df.copy()
display(df.iloc[:5,:11])
display(df.iloc[:5,11:])

D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,84
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,60
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,180
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,60
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,240


,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,4,2.0,0,0,1,0,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,2,2.0,0,0,1,0,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,7,1.0,0,0,1,0,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,2,1.0,0,0,1,0,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,14,1.0,7,7,1,0,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [3]:
df2 = df.iloc[:50, :]

In [4]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899164 entries, 0 to 899163
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   LoanNr_ChkDgt      899164 non-null  int64  
 1   Name               899150 non-null  object 
 2   City               899134 non-null  object 
 3   State              899150 non-null  object 
 4   Zip                899164 non-null  int64  
 5   Bank               897605 non-null  object 
 6   BankState          897598 non-null  object 
 7   NAICS              899164 non-null  int64  
 8   ApprovalDate       899164 non-null  object 
 9   ApprovalFY         899164 non-null  object 
 10  Term               899164 non-null  int64  
 11  NoEmp              899164 non-null  int64  
 12  NewExist           899028 non-null  float64
 13  CreateJob          899164 non-null  int64  
 14  RetainedJob        899164 non-null  int64  
 15  FranchiseCode      899164 non-null  int64  
 16  Ur

In [5]:
# 處理日期
d_cols = ['ChgOffDate', 'DisbursementDate', 'ApprovalDate']
df_copy[d_cols] = df_copy[d_cols].fillna('xx-xx-xx')

for col in d_cols:
    
    df_copy[col + '_year'] = df_copy[col].astype('str').apply(lambda x: x.split('-')[2])
    df_copy[col + '_month'] = df_copy[col].astype('str').apply(lambda x: x.split('-')[1])
    df_copy[col + '_day'] = df_copy[col].astype('str').apply(lambda x: x.split('-')[0])

df_copy.drop(d_cols, axis=1, inplace=True)
df_copy.drop('ApprovalFY', axis=1, inplace=True)

mon_dict = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12, 'xx': 'xx'}

# 月份文字轉數字
df_copy['ChgOffDate_month'] = df_copy['ChgOffDate_month'].map(mon_dict)
df_copy['DisbursementDate_month'] = df_copy['DisbursementDate_month'].map(mon_dict)
df_copy['ApprovalDate_month'] = df_copy['ApprovalDate_month'].map(mon_dict)

# 年份二位轉四位
def year_2d_to_4d(x):
    try:
        if int(x) > 14:  # 代表19xx年
            x = '19' + x
        else:  # 20xx年
            x = '20' + x
    except ValueError:
        pass
    return x

year_cols = ['ChgOffDate_year', 'DisbursementDate_year', 'ApprovalDate_year']

df_copy[year_cols] = df_copy[year_cols].astype('str').applymap(year_2d_to_4d)

In [6]:
def money_clean(org_money):
    org_money = org_money.strip().split('.')[0]
    org_money = org_money.strip('$').replace(',', '')

    return org_money

In [7]:
money_cols = ['DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']
df_copy.loc[:, money_cols] = df_copy.loc[:, money_cols].applymap(money_clean)
df_copy[money_cols] = df_copy[money_cols].astype('int64')

In [8]:
df_copy['NewExist'] = df_copy['NewExist'].map({2: 0, 1: 1})
df_copy['UrbanRural'] = df_copy['UrbanRural'].map({2: 0, 0: -1, 1: 1})
df_copy['RevLineCr'] = df_copy['RevLineCr'].map({'Y': 1, 'N': 0})
df_copy['LowDoc'] = df_copy['LowDoc'].map({'Y': 1, 'N': 0})

In [9]:
df_copy['Industry'] = df_copy['NAICS'].astype('str').apply(lambda x: x[:2])
df_copy['Industry'] = df_copy['Industry'].map({
    '11': 'Ag/For/Fish/Hunt',
    '21': 'Min/Quar/Oil_Gas_ext',
    '22': 'Utilities',
    '23': 'Construction',
    '31': 'Manufacturing',
    '32': 'Manufacturing',
    '33': 'Manufacturing',
    '42': 'Wholesale_trade',
    '44': 'Retail_trade',
    '45': 'Retail_trade',
    '48': 'Trans/Ware',
    '49': 'Trans/Ware',
    '51': 'Information',
    '52': 'Finance/Insurance',
    '53': 'RE/Rental/Lease',
    '54': 'Prof/Science/Tech',
    '55': 'Mgmt_comp',
    '56': 'Admin_sup/Waste_Mgmt_Rem',
    '61': 'Educational',
    '62': 'Healthcare/Social_assist',
    '71': 'Arts/Entertain/Rec',
    '72': 'Accom/Food_serv',
    '81': 'Other_no_pub',
    '92': 'Public_Admin',
    '0': 'Unknown'
})

df_copy.drop('NAICS', axis=1, inplace=True)

In [10]:
df_copy = df_copy.apply(lambda x: x.replace('xx', np.nan))

In [11]:
def na_check(df_data):
    data_na = (df_data.isnull().sum() / len(df_data))
    data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})

    return missing_data

In [12]:
na_check(df_copy)  # 7/20目標

,Missing Ratio
ChgOffDate_year,0.819055
ChgOffDate_month,0.819055
ChgOffDate_day,0.819055
RevLineCr,0.308597
LowDoc,0.006681
DisbursementDate_year,0.002634
DisbursementDate_month,0.002634
DisbursementDate_day,0.002634
MIS_Status,0.002221
BankState,0.001742


In [13]:
display(df_copy.iloc[:5,:18])
display(df_copy.iloc[:5,18:])

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,84,4,0.0,0,0,1,-1,0.0,1.0,60000,0
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,60,2,0.0,0,0,1,-1,0.0,1.0,40000,0
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,180,7,1.0,0,0,1,-1,0.0,0.0,287000,0
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,60,2,1.0,0,0,1,-1,0.0,1.0,35000,0
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,240,14,1.0,7,7,1,-1,0.0,0.0,229000,0


,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv,ChgOffDate_year,ChgOffDate_month,ChgOffDate_day,DisbursementDate_year,DisbursementDate_month,DisbursementDate_day,ApprovalDate_year,ApprovalDate_month,ApprovalDate_day,Industry
0,P I F,0,60000,48000,NaN,NaN,NaN,1999,2.0,28,1997,2,28,Retail_trade
1,P I F,0,40000,32000,NaN,NaN,NaN,1997,5.0,31,1997,2,28,Accom/Food_serv
2,P I F,0,287000,215250,NaN,NaN,NaN,1997,12.0,31,1997,2,28,Healthcare/Social_assist
3,P I F,0,35000,28000,NaN,NaN,NaN,1997,6.0,30,1997,2,28,Unknown
4,P I F,0,229000,229000,NaN,NaN,NaN,1997,5.0,14,1997,2,28,Unknown
